# Higgs Boson Discovery

## Problems with finding the invariant mass

You're probably coming here from the Physics StackOverflow question. I decided to add my own spin to some Higgs analysis, and ran into a little trouble. I am trying to get the invariant mass for each electron in the 2011 4e data, first making the attempt with one electron. My invariant masses are about 10 GeV away from the expected value of 0.0005 GeV.

### Part of the original description of the visualization

This is supposed to be a nice visualization to take me back to grad school - where I worked at [BNL](https://www.bnl.gov) / [RHIC](https://www.bnl.gov/rhic/) / [PHENIX](https://www.phenix.bnl.gov/) ([archived](https://web.archive.org/web/20230314100142/https://www.bnl.gov/world/) / [archived](https://web.archive.org/web/20230312202521/https://www.bnl.gov/rhic/) / [archived](https://web.archive.org/web/20230314191656/https://www.phenix.bnl.gov/)) which basically means I worked at a government particle accelerator. I wasn't at [CERN](https://home.cern/) / [LHC]() / [CMS](https://home.cern/science/experiments/cms) ([archived](https://web.archive.org/web/20230314192144/https://home.cern/) / [archived]() / [archived](https://web.archive.org/web/20230314192110/https://home.cern/science/experiments/cms)), which is one of the experiments contributing to the discovery of the Higgs boson and the provider of the data I'll be using. I did participate in _some_ collaboration with LHC groups. I was even invited to the [4<super>th</super> of July Discovery Announcement](https://en.wikipedia.org/wiki/Higgs_boson#Search_before_4_July_2012) ([archived](https://web.archive.org/web/20230314192829/https://en.wikipedia.org/wiki/Higgs_boson)), but I got my keys locked in my car while hanging out with friends halfway to Columbia University. (I think reality makes for a better story - we got in my car using tree branches, but it was long past the midnight announcement time.)

The results (when I accomplish my goal) should look something like what follows. The image comes from https://arxiv.org/format/1207.7235, the discovery paper for the Higgs.

<br/>
<div>
  <img src="./publication_4lepton_spectrum.png"
       alt="Publication histogram - our goal for the analysis is to be similar to this"
       width="400px">
</div>
<br/>

## Setup

Using `conda` for environment setup. This time, because of computer availability, I did it on the Windows CMD prompt (`Anaconda Prompt (miniconda3)`, specifically). `Jupyter` makes for simpler visualizations, so here I am.

<hr/>

### Virtual Environment And Getting From Git

If you're seeing the notebook simply displayed on GitHub and want to run it, this part will show you how. Note that you can also just run the commands from a normal command line. For doing the analysis with a Jupyter notebook, do one of the following 

1) Use MyBinder. (I'm going to try getting it set up with [MyBinder](https://mybinder.org/), but I haven't gotten that to work, yet. I'm pretty sure _you_ can still do it. If you figure it out, let me know.) 

2) Set up a virtual environment. Make a new folder (directory) for the project and go there.

#### Implementation of choice 2 through Python without conda //A//

On a normal `Python` setup (this one with Windows)

```
>mkdir higgs_venv
>python -m venv .\higgs_venv
>.\higgs_venv\Scripts\activate
(higgs_venv)>
```

#### Implementation of choice 2 through Python with conda //B//

On a `conda` setup (again, for this time, with Windows)

```
> conda create --name higgs_venv python=3.10
> conda activate higgs_venv
(higgs_venv)>
```

#### After either of the previous two ( //A// or //B//)

```
(higgs_venv)> git clone https://github.com/bballdave025/higgs_for_help.git
(higgs_venv)> cd higgs_for_help
```

### When are we? (good for experimental notebooks)

```
>powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]{5}_', '_'
1678797612_20230314T124012-0600

>::  Gosh, event that little extra makes me miss bash
```

<hr/>

### Environment stuff, pip installs, and maybe on to Jupyter 

Note that this needs to be done whether we're using Jupyter or not. If we're using Jupyter, you _must_ do this before starting the Jupyter server.

```
::  These next two commands can be done if you are trying 
::+ this on the command line, rather than from Jupyter
::+ or if you simply didn't do it earlier.
>conda create --name higgs_venv python=3.10
>conda activate higgs_venv
(higgs_venv)>
::  Do the following no matter what. This will need to be out on
::+ the command prompt in the `higgs_venv` environment.
(higgs_venv)>python -m pip install --upgrade pip
(higgs_venv)>pip install numpy
(higgs_venv)>pip install pandas
(higgs_venv)>pip install matplotlib
(higgs_venv)>pip install requests
(higgs_venv)>pip install pathlib
```

If you want to launch the Jupyter notebook from your own machine, also do the following from the command prompt.

**VERY IMPORTANT IF YOU'RE RUNNING THIS ON YOUR OWN MACHINE WITH JUPYTER**

```
(higgs_venv)>pip install jupyter
(higgs_venv)>jupyter notebook Higgs_Boson_Discovery_Visualization_Questions_for_Help.ipynb
```

That should take you to your browser, where you'll see the notebook ... and be able to interact with it.

Now, we can do our initial imports and image-dispay setup. 

### Imports and other setup

In [69]:
import numpy  as np
import pandas as pd
from matplotlib import pyplot as plt

## Next line only for Jupyter notebook.
%matplotlib inline

## Some Info (Physics)

Let's talk a little about the physics of the data we'll be looking at. You may know that Uranium can decay into lighter elements, as long as the combined mass-energy of the daughters (lighter elements' atoms, energy) is the same as the combined mass-energy - usually just the original mass - of the original Uranium.

The same happens for particles. The Higgs boson can decay into lighter particles and energy as long as its daughter decay particles and the energy is equal to the original mass-energy of the Higgs. One way that physics allows this to happen is for the Higgs to decay into four $\ell$eptons. (I use the script '$\ell$' to avoid confusion between '1' and 'l'.)

Here is a way physicists represent such a decay, which has the problems noted on the image.

<br/>
<div>
  <img src="./four_leptons_description_01.png"
       alt="Born (basically Feynman) diagrams for the two possible Higgs to four electrons scenario."
       width=75%>
</div>
<br/>

In case you're curious, I'll tell you that the the $Z^{(*)}$ lines are for off-mass-shell (also called off-shell or virtual) Z-bosons. You can look up details if you'd like. Actually, to be technically correct, the Higgs boson should be off-shell (virtual), too, and we should see it labeled $H^{(*)}$. Just for fun, I'll put in this image from the [Wikipedia article on the Higgs boson](https://en.wikipedia.org/wiki/Higgs_boson#Discovery_of_candidate_boson_at_CERN) ([archived](https://web.archive.org/web/20230314192829/https://en.wikipedia.org/wiki/Higgs_boson)). It should show up even if changes are made to the wiki article and or the image, because I'm using an [archived image](https://web.archive.org/web/20230314233304/https://en.wikipedia.org/wiki/File:4-lepton_Higgs_decay.svg). This diagram doesn't show whether something is off-shell/virtual with the $^{(*)}$, because one learns that anything in the middle of such a diagram is virtual/off-shell. Also, this next diagram shows only part of the interaction - the two protons have six valence quarks and gluons holding them together. At our collision energy, you would expect to find a lot of gluons.

<br/>
<div>
  <img src="https://web.archive.org/web/20230314233304/https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/4-lepton_Higgs_decay.svg/320px-4-lepton_Higgs_decay.svg.png"
       alt="Both sides of the diagram for Higgs to four leptons"
       width="480px">
</div>
<br/>

That represents to protons coming in and four leptons coming out.

### Three decay possibilites

In the first image, I said that we could simplify things while including more leptons than the electron. Let's use these symbols,

<br/>
<div>
  <img src="./four_leptons_description_02.png"
       alt="Symbols for electron, positron (positive version of the electron), muon, and antimuon (positive version of the muon)"
       width=300px>
</div>
<br/>

and give the three possibilites. We'll see these possibilities in the names of the `CSV` files.

**Possibility 1: ... $H^{(*)} \rightarrow \{ 2 Z^{(*)} \} \rightarrow \{ 4e \}$**<br/>
**Higgs to 4 electrons (via two Z bosons and after previous interactions)**

<br/>
<div>
  <img src="./decay_four_leptons_four_electrons.png"
       alt="Off-mass-shell (virtual) Higgs to two off-mass-shell (virtual) Z bosons to four electrons"
       width=300px>
</div>
<br/>

**Possibility 2: ... $H^{(*)} \rightarrow \{ 2 Z^{(*)} \} \rightarrow \{ 4\mu \}$**<br/>
**Higgs to four muons (via two Z bosons and after previous interactions)**

<br/>
<div>
  <img src="./decay_four_leptons_four_muons.png"
       alt="Off-mass-shell (virtual) Higgs to two off-mass-shell (virtual) Z bosons to four muons"
       width=300px>
</div>
<br/>

**Possibility 3: ... $H^{(*)} \rightarrow \{ 2 Z^{(*)} \} \rightarrow \{ 2e, 2\mu \}$**<br/>
**Higgs to 2 electrons and 2 muons (via two Z bosons and after previous interactions)**

<br/>
<div>
  <img src="./decay_four_leptons_two_electrons_two_muons.png"
       alt="Off-mass-shell (virtual) Higgs to two off-mass-shell (virtual) Z bosons to two electrons and two muons"
       width=300px>
</div>
<br/>

<hr/>

## Starting The Analysis

### URLs

In [70]:
data_csv_urls = []

# 4 electrons as daughter particles (two years' data)
url_4e_2011 = 'http://opendata.cern.ch/record/5200/files/4e_2011.csv'
data_csv_urls.append(url_4e_2011)
url_4e_2012 = 'http://opendata.cern.ch/record/5200/files/4e_2012.csv'
data_csv_urls.append(url_4e_2012)

# 4 muons as daughter particles (two years' data)
url_4mu_2011 = 'http://opendata.cern.ch/record/5200/files/4mu_2011.csv'
data_csv_urls.append(url_4mu_2011)
url_4mu_2012 = 'http://opendata.cern.ch/record/5200/files/4mu_2012.csv'
data_csv_urls.append(url_4mu_2012)

# 2 electrons and 2 muons as daughter particles (two years' data)
url_2e2mu_2011 = 'http://opendata.cern.ch/record/5200/files/2e2mu_2011.csv'
data_csv_urls.append(url_2e2mu_2011)
url_2e2mu_2012 = 'http://opendata.cern.ch/record/5200/files/2e2mu_2012.csv'
data_csv_urls.append(url_2e2mu_2012)

### Download using requests and shutil then reading in with pandas

#### Looking at one file

In [71]:
##  Before I loop through all of the events and particles, I'll just 
##+ load in the first one. That way, we can look more of the physics

##########################################
##  A peek at the data for 4e from 2011 ##
##+ Side scrolling will be necessary    ##
##########################################

import requests
import shutil
from pathlib import Path

my_url = 'http://opendata.cern.ch/record/5200/files/4e_2011.csv'
local_filename = my_url.split('/')[-1]

with requests.get(my_url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
        shutil.copyfileobj(r.raw, f)
    ##endof:  with open ... as f
##endof:  with requests ... as r

our_file = Path(local_filename)
we_have_our_file = ( our_file.is_file() )

assert(we_have_our_file)

No AssertionError! Hooray! We have our file!

I want to see if the file size was causing the timeout, or if it was (the most likely explanation) something with my local gateway.

For a discussion of the fastest and most-memory-friendly way to count lines in a file (especially big files), see [this note](#Line-Count-Discussion) near the end of the notebook.

##### Line-count functions

In [72]:
def _make_count_generator(reader):
    b = reader(1024 * 1024)
    while b:
        yield b
        b = reader(1024*1024)
    ##endof:  while b
##endof:  def _make_count_generator(reader)


#  DWB note: 20230316T164800-0600
#+ I don't know what will happen with an empty file
def raw_generator_line_count(filename):
    with open(filename, 'rb') as fp:
        my_generator = _make_count_generator(fp.raw.read)
        count = sum(buffer.count(b'\n') for buffer in my_generator)
    ##endof:  with open ... fp
    
    return count + 1
    
##endof:  def raw_generator_line_count(filename)

##### Line count for file

In [73]:
n_lines = raw_generator_line_count(local_filename)
print("\nFile: " + str(local_filename) + "\nhas " + str(n_lines) + " lines.")


File: 4e_2011.csv
has 9 lines.


Well, that was anticlimactic. The output was

```
File: 4e_2011.csv
has 9 lines.
```

#### Let's have some physics fun with the first batch of data

After all, we can visualize it all. As I was saying, some side scrolling will be needed, so we need to change some `pandas` stuff.

In [74]:
pd.options.display.max_columns = 50

# Get our dataframe
my_4e2011_df = pd.read_csv(local_filename)

#my_4e2011_df.describe(include="all") #  That will give you a scrollable,
                                      #+ HTML-type table

print(my_4e2011_df.describe(include="all")) # Just plain-text output

                 Run         Event       PID1          E1        px1  \
count       7.000000  7.000000e+00   7.000000    7.000000   7.000000   
mean   169313.142857  3.351673e+08   1.571429  132.185000 -21.429387   
std      4273.231620  3.750184e+08  11.759495  113.218065  34.336417   
min    163659.000000  1.034711e+07 -11.000000   46.296700 -58.328400   
25%    165716.500000  6.511811e+07 -11.000000   55.682150 -47.917400   
50%    171106.000000  1.419548e+08  11.000000   87.628000 -19.222600   
75%    172875.500000  5.934871e+08  11.000000  163.954500  -9.188755   
max    173243.000000  8.766590e+08  11.000000  352.097000  41.757600   

              py1         pz1         pt1      eta1      phi1        Q1  \
count    7.000000    7.000000    7.000000  7.000000  7.000000  7.000000   
mean    23.073559   81.730426   55.990400  0.632904 -0.411380 -0.142857   
std     45.076298  145.060001   26.652413  1.347241  2.429490  1.069045   
min    -15.448800  -76.166700   32.073500 -1.329820

That might be more information than we get by looking at the whole dataset with `head`.

In [75]:
#out-for-now#my_4e2011_df.head(10) #  That will give you a scrollable,
                                   #+ HTML-type table

print(my_4e2011_df.head(10)) #  Just plain-text output, there are less
                             #+ than ten events, so we will see all of
                             #+ the data

      Run      Event  PID1        E1       px1        py1        pz1  \
0  167675  876658967   -11   46.2967 -45.47210   -8.61010   -1.24072   
1  173243   16706390    11  352.0970 -58.32840   -8.93069  347.11700   
2  171106  141954801    11   49.6757  -8.98850   44.24350  -20.72180   
3  163758  113529826   -11  112.0390 -19.22260   25.67490  107.35000   
4  172952  842265702   -11   87.6280  41.75760   11.55910  -76.16670   
5  163659  344708580    11   61.6886 -50.36270  -15.44880   32.10020   
6  172799   10347106    11  215.8700  -9.38901  113.02700  183.67500   

        pt1      eta1      phi1  Q1  PID2        E2       px2        py2  \
0   46.2801 -0.026806 -2.954460   1    11   45.8568  43.53670  14.370800   
1   59.0081  2.472280 -2.989660  -1   -11  124.2200  55.88820  -7.347430   
2   45.1473 -0.444228  1.771230  -1    11   68.4708  41.22040  -0.639024   
3   32.0735  1.922810  2.213460   1    11   26.6863   6.94914 -23.053800   
4   43.3279 -1.329820  0.270053   1    11  

<hr/>

In particle physics, we have something called the [invariant mass](https://en.wikipedia.org/wiki/Invariant_mass) ([archived](https://web.archive.org/web/20230313220738/https://en.wikipedia.org/wiki/Invariant_mass)). The invariant mass in such a collision - when talking about daughter particles - is the mass of the particle that decayed. It's the big '`M`' in the last column. (If you don't understand that, don't worry - just forget I said anything.)

There are a lot of different masses in that last column. Consider that an electron has a mass of about $0.5 MeV/c^2$, which is the same as $0.0005 GeV/c^2$. (We'll get back to that) 

***&lt;part-you-can-skip&gt;***<br/>
Particle physicists, again with our own quirky little ways, use a system called [natural units](https://en.wikipedia.org/wiki/Natural_units#Natural_units_(particle_and_atomic_physics)) ([archived](https://web.archive.org/web/20230316232513/https://en.wikipedia.org/wiki/Natural_units)). Some will say that this allows us to write $c$ (the speed of light) as $1$. That's oversimplifying it (it's more like making the physical-constant coefficients equal to $1$, in this case by making the length unit the same as the time unit ... -ish). Let's just say it lets us get away with writing equations and measurements without including the defining constants, which include $c$. If you want to know more, you can also look up [nondimensionalization](https://en.wikipedia.org/wiki/Nondimensionalization) ([archived](https://web.archive.org/web/20230316040337/https://en.wikipedia.org/wiki/Nondimensionalization))); I'm guessing you probably _don't_ want to know more, or even as much as I've said.<br/>
***&lt;/part-you-can-skip&gt;***

All of that to say that we can approximate the masses of the electron and the Higgs as 

`{mass_electron, mass_higgs} = {0.5 MeV, 126000 MeV} = {0.0005 GeV, 126 GeV}`

Any way you look at it, that Higgs is a _lot_ more massive than four electrons. Where does the extra mass go? Well, some of the mass gets converted into energy (remember $E = m c^2$), a lot of it being energy of motion. 

The general formula for invariant mass, $m_0$, for one particle in this case, is

$m_0 = \sqrt{E^2 - \lvert \textbf{p} \rvert^2}$

Note that this is in natural units. The value of $c$ can be considered as $1$ (in speed-of-light units).

<hr/>

Let's look at the mass issue in two ways, both using the invariant mass. 

First, I've tried looking at each electron's invariant mass. I've had some problems with that, and things don't match. I'll be taking the question to the Physics StackExchange site. 

As somewhat of a sanity check, I'll find the invariant mass of the particle that decayed into four leptons (four electrons in the case we're doing). 

We'll take a look at one event to check things out.

Here's some code to let you see the info we get from one event - basically the headers and the values for the first event.

In [76]:
## Adding 2023-03-16
import re

table_of_tables = []
current_table = []
is_a_new_table = True
is_the_first_element = True

for my_header in my_4e2011_df.columns.values.tolist():
    if ( re.match(r"PID\d+", my_header) or
         re.match(r"mZ1", my_header) or
         re.match(r"M", my_header)):
        is_a_new_table = True
    ##endof:  if <and all the re.match>
    
    if is_a_new_table:
        if not is_the_first_element:
            table_of_tables.append(current_table)
        else:
            is_the_first_element = False
        ##endof:  if/else is_not_the_first_element
        
        current_table = [my_header]
        is_a_new_table = False
        
    else:
        current_table.append(my_header)
    ##endof:  if/else is_a_new_table
    
##endof:  for my_header in my_4e2011_df.columns.values.tolist()

table_of_tables.append(current_table)

In [77]:
import pprint

pprint.pprint(table_of_tables)

[['Run', 'Event'],
 ['PID1', 'E1', 'px1', 'py1', 'pz1', 'pt1', 'eta1', 'phi1', 'Q1'],
 ['PID2', 'E2', 'px2', 'py2', 'pz2', 'pt2', 'eta2', 'phi2', 'Q2'],
 ['PID3', 'E3', 'px3', 'py3', 'pz3', 'pt3', 'eta3', 'phi3', 'Q3'],
 ['PID4', 'E4', 'px4', 'py4', 'pz4', 'pt4', 'eta4', 'phi4', 'Q4'],
 ['mZ1', 'mZ2'],
 ['M']]


The output was

```
[['Run', 'Event'],
 ['PID1', 'E1', 'px1', 'py1', 'pz1', 'pt1', 'eta1', 'phi1', 'Q1'],
 ['PID2', 'E2', 'px2', 'py2', 'pz2', 'pt2', 'eta2', 'phi2', 'Q2'],
 ['PID3', 'E3', 'px3', 'py3', 'pz3', 'pt3', 'eta3', 'phi3', 'Q3'],
 ['PID4', 'E4', 'px4', 'py4', 'pz4', 'pt4', 'eta4', 'phi4', 'Q4'],
 ['mZ1', 'mZ2'],
 ['M']]
```

Note that we can use `my_4e2011_df.info()` to get even more info about these (not about their physics, but about the data types, count, etc. I've done that in a [note near the end](#More-Dataframe-Info).

<hr/> <hr/>

### One daughter particle

For our one-particle invariant mass, let's look back to the structure of the data we're inspecting.

```
[['Run', 'Event'],
 ['PID1', 'E1', 'px1', 'py1', 'pz1', 'pt1', 'eta1', 'phi1', 'Q1'],
 ['PID2', 'E2', 'px2', 'py2', 'pz2', 'pt2', 'eta2', 'phi2', 'Q2'],
 ['PID3', 'E3', 'px3', 'py3', 'pz3', 'pt3', 'eta3', 'phi3', 'Q3'],
 ['PID4', 'E4', 'px4', 'py4', 'pz4', 'pt4', 'eta4', 'phi4', 'Q4'],
 ['mZ1', 'mZ2'],
 ['M']]
```

It it is now pretty easy to see how we can do $E^2$. For the $\lvert \textbf{p} \rvert^2$, we use a standard vector length formula (Pythagorean theorem). I'm not sure what to do with `ptN`, which I'm pretty sure is transverse momentum. That's basically a different way of measuring and showing the momentum from $x$ and $y$. (If I recall correctly, it's mostly used for minding particles such as neutrinos.) We can use our Pythagorean Theorem for the norm of the momentum vector using $x,y,z$. So, for each particle, (1,2,3,4), for which we have the components of momentum (we'll designate each particle with $N \in \{1, 2, 3, 4\}$)

$\lvert \textbf{p} \rvert = \sqrt{(pxN)^2 + (pyN)^2 + (pzN)^2}$

We'll do this with our dataframe, and to be a bit simpler (not as busy on the page), we'll do it one particle at a time. For the first try, we'll do it one event at a time as well.

For one particle, in any frame of reference (ignore that any-frame-of-reference part if it confuses you), the invariant mass is given by

$m_0 = \sqrt{E^2 - \lvert \textbf{p} \rvert^2}$

We'll check that each of the electrons has the correct (invariant) mass.

#### Invariant mass for each particle

We'll use our equation, rewritten as

$m_0 = \sqrt{E^2 - \left[(pxN)^2 + (pyN)^2 + (pzN)^2\right]}$ with $N \in {1, 2, 3, 4}$

Note that this will give us four different $m_0$ values, _NOT_ a sum over $N$.

In [95]:
## added 2023-03-17
from math import sqrt
## added 2023-03-18
import sys

do_debug = False

def invariant_mass_from_values(energy, px, py, pz, 
                               do_avoid_imaginary=True,
                               do_display_invariant_mass=True,
                               do_print_if_p2_gt_e2=True):
    '''
    :brief: Calculates the invariant mass of a particle
    
    :param energy:  The energy of the particle
    :param px:      The x-component of the particle's momentum
    :param py:      The y-component of the particle's momentum
    :param pz:      The z-component of the particle's momentum
    
    :returns invariant_mass: The invariant mass (rest mass)
    '''
    
    invariant_mass = None
    
    #  The values come to us from a pandas dataframe. We need
    #+ to cast each into just a float. This function might
    #+ only work when we have one row. -DWB 2023-03-18
    
    energy = float(energy)
    px = float(px)
    py = float(py)
    pz = float(py) 
                   #  Oh! Hahahahaha! What a bug!
                   #+ Seriously?!? This whole time, 
                   #+ the problem has been `pz = float(py)`
                   #+ That tells me I'm a programmer.
                   #+ I started this about 2023-03-14,
                   #+ and started doing the single e
                   #+ stuff around 2023-03-17, and I have
                   #+ it now 20230321T172100-0600
    
    if do_debug:
        print("\n##DEBUG 1##")
        print("energy: " + str(energy))
        print("px: " + str(px))
        print("py: " + str(py))
        print("pz: " + str(pz))
    ##endof:  if debug
    
    #  For avoiding imaginary numbers, we're going to use the 
    #+ following, depending on the sign of E^2 - |p|^2.
    #+ Remember that the square root of a negative number
    #+ is imaginary. The energy of an e+ (positron) can be
    #+ viewed as negative, or imaginary in some cases (?)
    #+                PLEASE COMMENT ON THIS, PHYSICS STACKEXCHANGE PEOPLE
    #+
    #+ so, we will have
    #+
    #+  (+/-) sqrt( (+/-) (E^2 - |p|^2) )
    #+    ^           ^
    #+    |           \
    #+                 ---------
    #+  Calling this            \
    #+ `total_sign_transform`    \
    #+                           |
    #+                 Calling this
    #+                 `e2_minus_p2_sign_transform`
    
    e_squared_part = (energy ** 2)
    norm_p_squared_part = ((px ** 2) + (py ** 2) + (pz ** 2))
    
    if do_debug:
        print("\n##DEBUG 2##")
        print("e_squared_part: " + str(e_squared_part))
        print("norm_p_squared_part: " + str(norm_p_squared_part))
    ##endof:  if do_debug
    
    #  remember that the time-component of the vector-length
    #+ formula has a different sign
    e2_minus_p2 = e_squared_part - norm_p_squared_part
    
    if do_debug:
        print("\n##DEBUG 3##")
        print("e2_minus_p2: " + str(e2_minus_p2))
    ##endof:  if do_debug
    
    total_sign_transform = 1       # plus 1 => no change
    e2_minus_p2_sign_transform = 1 # plus 1 => no change
    
    radicand_is_negative = ( e2_minus_p2 < 0 )
    
    if radicand_is_negative and do_avoid_imaginary:
        if do_print_if_p2_gt_e2:
            print("NOTE: The radicand had a negative value, " + \
                  "so we're adjusting signs.")
        ##endof:  if do_print_if_p2_gt_e2
        
        total_sign_transform = -1
        e2_minus_p2_sign_transform = -1
    ##endof:  if radicand_is_negative
    
    try:
        new_radicand = e2_minus_p2_sign_transform * e2_minus_p2
        invariant_mass = total_sign_transform * sqrt( new_radicand )
        invariant_mass = float(invariant_mass)
        if do_debug:
            print("\n##DEBUG 3##")
            print("e2_minus_p2: " + str(e2_minus_p2))
        ##endof:  if do_debug
    except Exception as ve:
        invariant_mass = "NaN"
        print("\nYour E^2 - |p|^2 is negative, so taking", file=sys.stderr)
        print("its square root will lead to an imaginary", file=sys.stderr)
        print("number. Not very physical (in this case,", file=sys.stderr)
        print("unless there's something with antiparticles", 
              file=sys.stderr)
        print("or something giving complex values that", file=sys.stderr)
        print("can somehow be used to show the invariant", file=sys.stderr)
        print("the mass of the particle. [?]).\n", file=sys.stderr)
        print("The details of the exception are:", file=sys.stderr)
        print(repr(ve), file=sys.stderr)
        print("\nNaN will be returned for the invariant mass.\n",
              file=sys.stderr)
        print("You might want to look at changing the", file=sys.stderr)
        print("do_avoid_imaginary parameter to True in", file=sys.stderr)
        print("the invariant_mass_from_values function.\n", file=sys.stderr)
    finally:
        pass
    ##endof:  try/catch/finally
    
    if do_display_invariant_mass:
        print("invariant_mass: " + str(invariant_mass))
    ##endof:  if do_display_invariant_mass
    
    return invariant_mass

##endof:  def invariant_mass_from_values(<params>)

<hr/>

In [90]:
do_display_first = False #  You can change this depending on the amount
                         #+ of output you want.

print("With do_debug=True")

print()

if do_display_first:
    print(my_4e2011_df.iloc[1])
##endof:  if do_display_first

#  archived_ref="https://web.archive.org/web/" + \
#+ "20230318225210/https://stackoverflow.com/questions/" + \
#+ "12021754/how-to-slice-a-pandas-dataframe-by-position"
first_event_df = my_4e2011_df[:1]

event_1_particle_1_df = \
  first_event_df.assign(
            Invariant_Mass_1 = lambda x: \
               invariant_mass_from_values(x.E1,
                                          x.px1,
                                          x.py1,
                                          x.pz1)
) 

#event_1_particle_1_df.loc[:, ['E1', 'px1', 'py1', 'pz1', 'Invariant_Mass_1']]

print()

print("RESULTS:\n" + \
  str(event_1_particle_1_df.loc[:, ['E1', 'px1', 'py1', 'pz1', 'Invariant_Mass_1']])
)

With do_debug=True


##DEBUG 1##
energy: 46.2967
px: -45.4721
py: -8.6101
pz: -8.6101

##DEBUG 2##
e_squared_part: 2143.38443089
norm_p_squared_part: 2215.97952243

##DEBUG 3##
e2_minus_p2: -72.59509153999988
NOTE: The radicand had a negative value, so we're adjusting signs.

##DEBUG 3##
e2_minus_p2: -72.59509153999988
invariant_mass: -8.52027532066892

RESULTS:
        E1      px1     py1      pz1  Invariant_Mass_1
0  46.2967 -45.4721 -8.6101 -1.24072         -8.520275


In [96]:
wawa = 46.2967**2
wuwu = ((-45.4721)**2 + (-8.6101)**2 + (-1.24072)**2)
print(wawa)
print(wuwu)
wywy=-((wuwu-wawa)**(0.5))
print(wywy)

print()
print("Why didn't I look at it this simply before?")
print("Because I trusted it to do my calculations.")
print("And it did them, perfectly. Perfectly as I")
print("had written them, which was the wrong way!")
print("I'm now laughing, after about a week.")

2143.38443089
2143.3850865384
-0.02560563219235628

Why didn't I look at it this simply before?
Because I trusted it to do my calculations.
And it did them, perfectly. Perfectly as I
had written them, which was the wrong way!
I'm now laughing, after about a week.


Looking for some insight on the problem of my calculated electron invariant masses being off.

All I get from those previous commands are

(from just `event_1_particle_1_df.loc[:, ['E1', 'px1', 'py1', 'pz1', 'Invariant_Mass_1']]`)

|   |E1 |px1|py1|pz1|Invariant_Mass_1|
|--:|--:|--:|--:|--:|---------------:|
|**0**|46.2967|-45.4721|-8.6101|-1.24072|-8.520275|

OR

(from the `print` command)

```
        E1      px1     py1      pz1  Invariant_Mass_1
0  46.2967 -45.4721 -8.6101 -1.24072         -8.520275
```

That shows our electron invariant mass to be -8.520275 (GeV). I can live with the invariant mass being negative. I think that means it's an antiparticle (positron, a.k.a. anti-electron, in this case). However, our invariant mass should, in that case, be -0.0005. (The invariant masses of the Z-bosons and the Higgs show up in GeV - natural units again - so I would expect that of the electron to do the same. I don't know how the invariant masses of the Z-bosons would be close to correct, but those of the electrons wouldn't be correct. The electrons are observed, so they shouldn't be off-mass-shell at all. I think I'll at least try with the other three particles. If they all seem weird, it will be off to Physics StackExchange.

<hr/>

In [97]:
first_event_df = my_4e2011_df[:1]

event_1_particle_1_df = \
  first_event_df.assign(
            Invariant_Mass_1 = lambda x: \
               invariant_mass_from_values(x.E1,
                                          x.px1,
                                          x.py1,
                                          x.pz1)
) 

e1p1 = event_1_particle_1_df.loc[:, ['E1', 'px1', 'py1', 'pz1', 'Invariant_Mass_1']]

event_1_particle_2_df = \
  first_event_df.assign(
            Invariant_Mass_2 = lambda x: \
               invariant_mass_from_values(x.E2,
                                          x.px2,
                                          x.py2,
                                          x.pz2)
)

e1p2 = event_1_particle_2_df.loc[:, ['E2', 'px2', 'py2', 'pz2', 'Invariant_Mass_2']]

event_1_particle_3_df = \
  first_event_df.assign(
            Invariant_Mass_3 = lambda x: \
               invariant_mass_from_values(x.E3,
                                          x.px3,
                                          x.py3,
                                          x.pz3)
)

e1p3 = event_1_particle_3_df.loc[:, ['E3', 'px3', 'py3', 'pz3', 'Invariant_Mass_3']]

event_1_particle_4_df = \
  first_event_df.assign(
            Invariant_Mass_4 = lambda x: \
               invariant_mass_from_values(x.E4,
                                          x.px4,
                                          x.py4,
                                          x.pz4)
)

e1p4 = event_1_particle_4_df.loc[:, ['E4', 'px4', 'py4', 'pz4', 'Invariant_Mass_4']]

#print("As an example: `type(e1p1)`: `" + str(type(e1p1)) + \
#      "`\ngives OUTPUT `<class 'pandas.core.frame.DataFrame'>`")

NOTE: The radicand had a negative value, so we're adjusting signs.
invariant_mass: -8.52027532066892
NOTE: The radicand had a negative value, so we're adjusting signs.
invariant_mass: -14.340081099143065
NOTE: The radicand had a negative value, so we're adjusting signs.
invariant_mass: -14.73344795117559
NOTE: The radicand had a negative value, so we're adjusting signs.
invariant_mass: -6.978215136852907


In [98]:
print(e1p1)
print("-----")
print(e1p2)
print("-----")
print(e1p3)
print("-----")
print(e1p4)

        E1      px1     py1      pz1  Invariant_Mass_1
0  46.2967 -45.4721 -8.6101 -1.24072         -8.520275
-----
        E2      px2      py2       pz2  Invariant_Mass_2
0  45.8568  43.5367  14.3708 -0.940301        -14.340081
-----
        E3      px3      py3      pz3  Invariant_Mass_3
0  27.2561 -13.6734 -19.6597  13.0164        -14.733448
-----
        E4       px4      py4      pz4  Invariant_Mass_4
0  7.57191 -0.091305  7.28083 -2.07727         -6.978215


Here's what the output comes out to be.

```
        E1      px1     py1      pz1  Invariant_Mass_1
0  46.2967 -45.4721 -8.6101 -1.24072         -8.520275
-----
        E2      px2      py2       pz2  Invariant_Mass_2
0  45.8568  43.5367  14.3708 -0.940301        -14.340081
-----
        E3      px3      py3      pz3  Invariant_Mass_3
0  27.2561 -13.6734 -19.6597  13.0164        -14.733448
-----
        E4       px4      py4      pz4  Invariant_Mass_4
0  7.57191 -0.091305  7.28083 -2.07727         -6.978215
```

(Well, apparently a negative sign on the invariant mass can't mean it's an antiparticle, or we just violated charge conservation and lepton number conservation and probably some other stuff as we consider the daughters of the Higgs.)

They're all off, all the invariant mass numbers. Maybe the electron invariant mass is too small to be found from such an analysis. Maybe they're within the size of the error bars. More likely, there's something I'm missing or doing wrong somewhere. That's good. It means I'm learning stuff.

I do wonder if particle 2 and particle 3 were from the same Z-boson, since whatever we calculated is close for those two. I don't know. I would imagine that they would put particles 1 and 2 as the ones from one $Z_0$, and particles 3 and 4 as those from the other. There's no _a priori_ reason to assume that, though.

<hr/>

### The parent particle

The general formula for invariant mass, $m_0$, for one particle in this case, is

$m_0 = \sqrt{E^2 - \lvert \textbf{p} \rvert^2}$

In a particle decay, the invariant mass of the the system of particle(s) stays the same - the invariant mass calculated using the energy and momentum of the daughter particles is equal to the invariant mass of the parent particle. However, we can't just find the equivalent mass for the first daughter particle, then the second daughter particle, etc. We need to look at them all together. This is shown in the general formula for the invariant mass of the system of daughter particles, usually notated as $W$, and which is equal to the invariant mass of the parent particle,

$W = \sqrt{ \left( \sum{E} \right)^2 - \left( \lvert \sum{\textbf{p}} \rvert \right)^2}$

Note that this, just like the general formula for invariant mass, is in natural units.

The way vectors work makes this nice for us. We can write the equation using only the values in our list.

$W = \sqrt{\left( \sum_N{E} \right)^2 - \sum_N{ \left[ (pxN)^2 + (pyN)^2 + (pzN)^2 \right]}}$ with $N \in \{1, 2, 3, 4\}$

or, even more exhaustively,

$W = \sqrt{(} \overline{ E1 + E2 + E3 + E4 )^2 - ... }$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\overline{ ... {[} (px1)^2 + (px2)^2 + (px3)^2 + (px4)^2 + ... }$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\overline{ ... (py1)^2 + (py2)^2 + (py3)^2 + (py4)^2 + ... }$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\overline{ ... (pz1)^2 + (pz2)^2 + (pz3)^2 + (pz4)^2 ] }$ 

In [99]:
#  Let's start by assuming that our E^2 part will be
#+ larger than our |p|^2 part
#+ ... the assumption seems borne out.

checking_system_invariant_masses_df = \
  my_4e2011_df.assign(
      W = lambda ev: (
        ( ( (ev.E1 + ev.E2 + ev.E3 + ev.E4) ** 2) -
          ( (ev.px1 + ev.px2 + ev.px3 + ev.px4) ** 2 +
            (ev.py1 + ev.py2 + ev.py3 + ev.py4) ** 2 +
            (ev.pz1 + ev.pz2 + ev.pz3 + ev.pz4) ** 2  )
        ) ** 0.5 
   )
)

events_inv_masses = checking_system_invariant_masses_df.loc[:, ['M', 'W']]

print(events_inv_masses)

          M           W
0  125.5280  125.528108
1  285.9600  285.960010
2  186.9150  186.915162
3  100.9230  100.922540
4   95.4259   95.425992
5  138.1930  138.193081
6  363.5380  363.537956


The results match wonderfully to within rounding error. Here is the output.

```
          M           W
0  125.5280  125.528108
1  285.9600  285.960010
2  186.9150  186.915162
3  100.9230  100.922540
4   95.4259   95.425992
5  138.1930  138.193081
6  363.5380  363.537956
```

I'll check one more thing. I'll send the values through my function. My function was made for values, though, so I'm going to have to deal with it. I'm not sure how to do so given the fact that `Series` objects get passed to my function. I think it has to do with the fact that I'm sending different column values into my lambda-ed function, rather than just all the values. [This](https://stackoverflow.com/a/55190150/6505499) ([archived](https://web.archive.org/web/20230321215636/https://stackoverflow.com/questions/20829748/pandas-assigning-multiple-new-columns-simultaneously)) or [that](https://stackoverflow.com/a/43668425/6505499) ([archived](https://web.archive.org/web/20230321220603/https://stackoverflow.com/questions/43667934/add-multiple-calculated-columns-to-a-pandas-dataframe-at-once)) might hold some answers.

In [100]:
do_this_method_and_see_errors = False
if do_this_method_and_see_errors:
    check_system_invariant_masses_func_df = \
    my_4e2011_df.assign(
          W = lambda ev:
              ( invariant_mass_from_values(
                  energy=(ev.E1 + ev.E2 + ev.E3 + ev.E4),
                  px=(ev.px1 + ev.px2 + ev.px3 + ev.px4),
                  py=(ev.py1 + ev.py2 + ev.py3 + ev.py4),
                  pz=(ev.pz1 + ev.pz2 + ev.pz3 + ev.pz4))
              ), axis=1
    )##endof:  my_4e2011_df.assign
    
    events_inv_masses_func = \
      check_system_invariant_masses_func_df.loc[:, ['M', 'W']]

    print("RESULTS")
    print(events_inv_masses_func)
##endof:  if do_this_method_and_see_errors

The only way I can think of to pass the values to my function is to iterate over the rows. That could cause a firestorm, such as the one seen [here](https://web.archive.org/web/20230321202813/https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas). I do want to look deeper into Pandas to figure that out, but that's not what I'm trying to figure out, now.

In [101]:
for index, row in my_4e2011_df.iterrows():
    total_E  = row.E1  + row.E2  + row.E3  + row.E4
    total_px = row.px1 + row.px2 + row.px3 + row.px4
    total_py = row.py1 + row.py2 + row.py3 + row.py4
    total_pz = row.pz1 + row.pz2 + row.pz3 + row.pz4
    
    my_W = invariant_mass_from_values(total_E, 
                                      total_px, 
                                      total_py, 
                                      total_pz,
                                      do_display_invariant_mass=False)
    my_M = row.M
    
    print(str(my_M), str(my_W), str(my_M-my_W))
##endof:  for

125.528 125.6591033027505 -0.1311033027504891
285.96 673.8486296031871 -387.8886296031871
186.915 197.14048963166832 -10.225489631668324
100.923 161.01746813448972 -60.094468134489716
95.4259 155.9105773183437 -60.484677318343714
138.193 148.9299069063654 -10.736906906365391
363.538 414.52724145289164 -50.98924145289163


The result (without_the_debugging) was

```
125.528 125.6591033027505 -0.1311033027504891
285.96 673.8486296031871 -387.8886296031871
186.915 197.14048963166832 -10.225489631668324
100.923 161.01746813448972 -60.094468134489716
95.4259 155.9105773183437 -60.484677318343714
138.193 148.9299069063654 -10.736906906365391
363.538 414.52724145289164 -50.98924145289163
```

There's a problem with the function.

Yes, there was. Right at the start. Sneaked in when I wanted it to do just the first event. `pz1 = float(py1)`. That's a whole different number. Well, I feel like I've learned a lot about `pandas`, anyway.

<hr/>


## Line Counting Discussion

### We could call it Appendix B

I think the best discussion of the most optimized line-counting algorithm is at

```
lc_ref_1="https://stackoverflow.com/questions/845058/" + \
"how-to-get-line-count-of-a-large-file-cheaply-in-python/27518377#27518377"
archived_lc_ref_1="https://web.archive.org/web/20230316221423/" + \
"https://stackoverflow.com/questions/845058/" + \
"how-to-get-line-count-of-a-large-file-cheaply-in-python/27518377#27518377"
```

Look for the answer from @Michael_Bacon, then you can read the answer by @Ryan_Ginstrom to give some background. I would go straight to the archived version if you're searching for those answers - that's why I archived it.

The answer by adds generator/buffer solutions, not discussed by @Ryan_Ginstrom, but one of which is also discussed in another good source,

```
lc_ref_2="https://pynative.com/python-count-number-of-lines-in-file/"
archived_lc_ref_2="https://web.archive.org/web/20230316215432/" + \
"https://pynative.com/python-count-number-of-lines-in-file/"
```

There's also a decent discussion from geeksforgeeks, which includes some Big Oh discussion, but it's not as comprehensive as the other two and doesn't discuss the generator stuff.

```
lc_ref_3="https://www.geeksforgeeks.org/count-number-of-lines-" + \
"in-a-text-file-in-python/"
archived_anchor_ref="https://web.archive.org/web/20230306171715/" + \
"https://www.geeksforgeeks.org/count-number-of-lines-" + \
"in-a-text-file-in-python/"
```

<hr/>

## More Dataframe Info

### We could call it Appendix C

In [88]:
my_4e2011_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Run     7 non-null      int64  
 1   Event   7 non-null      int64  
 2   PID1    7 non-null      int64  
 3   E1      7 non-null      float64
 4   px1     7 non-null      float64
 5   py1     7 non-null      float64
 6   pz1     7 non-null      float64
 7   pt1     7 non-null      float64
 8   eta1    7 non-null      float64
 9   phi1    7 non-null      float64
 10  Q1      7 non-null      int64  
 11  PID2    7 non-null      int64  
 12  E2      7 non-null      float64
 13  px2     7 non-null      float64
 14  py2     7 non-null      float64
 15  pz2     7 non-null      float64
 16  pt2     7 non-null      float64
 17  eta2    7 non-null      float64
 18  phi2    7 non-null      float64
 19  Q2      7 non-null      int64  
 20  PID3    7 non-null      int64  
 21  E3      7 non-null      float64
 22  px3   

The output was

```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Run     7 non-null      int64  
 1   Event   7 non-null      int64  
 2   PID1    7 non-null      int64  
 3   E1      7 non-null      float64
 4   px1     7 non-null      float64
 5   py1     7 non-null      float64
 6   pz1     7 non-null      float64
 7   pt1     7 non-null      float64
 8   eta1    7 non-null      float64
 9   phi1    7 non-null      float64
 10  Q1      7 non-null      int64  
 11  PID2    7 non-null      int64  
 12  E2      7 non-null      float64
 13  px2     7 non-null      float64
 14  py2     7 non-null      float64
 15  pz2     7 non-null      float64
 16  pt2     7 non-null      float64
 17  eta2    7 non-null      float64
 18  phi2    7 non-null      float64
 19  Q2      7 non-null      int64  
 20  PID3    7 non-null      int64  
 21  E3      7 non-null      float64
 22  px3     7 non-null      float64
 23  py3     7 non-null      float64
 24  pz3     7 non-null      float64
 25  pt3     7 non-null      float64
 26  eta3    7 non-null      float64
 27  phi3    7 non-null      float64
 28  Q3      7 non-null      int64  
 29  PID4    7 non-null      int64  
 30  E4      7 non-null      float64
 31  px4     7 non-null      float64
 32  py4     7 non-null      float64
 33  pz4     7 non-null      float64
 34  pt4     7 non-null      float64
 35  eta4    7 non-null      float64
 36  phi4    7 non-null      float64
 37  Q4      7 non-null      int64  
 38  mZ1     7 non-null      float64
 39  mZ2     7 non-null      float64
 40  M       7 non-null      float64
dtypes: float64(31), int64(10)
memory usage: 2.4 KB
```